# Calculate Biomass from Non-tree Vegetation and Litter

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import os
import pandas as pd
import numpy as np

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import DATA_DIR, GCP_PROJ_ID, CARBON_POOLS_OUTDIR, CARBON_STOCK_OUTDIR

from src.biomass_equations import vmd0003_eq1

In [3]:
# Variables
NTV_LITTER_CSV = CARBON_POOLS_OUTDIR / "saplings_ntv_litter.csv"
PLOT_INFO_CSV = CARBON_POOLS_OUTDIR / "plot_info.csv"

# BigQuery Variables
DATASET_ID = "carbon_stock"
IF_EXISTS = "replace"

## Load data

In [4]:
if PLOT_INFO_CSV.exists():
    plot_info = pd.read_csv(PLOT_INFO_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.plot_info"""

    # Read the BigQuery table into a dataframe
    plot_info = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    plot_info.to_csv(PLOT_INFO_CSV, index=False)

In [5]:
plot_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_id               674 non-null    object 
 1   data_recorder           674 non-null    object 
 2   team_no                 674 non-null    int64  
 3   plot_code_nmbr          674 non-null    int64  
 4   plot_type               674 non-null    object 
 5   sub_plot                674 non-null    object 
 6   yes_no                  674 non-null    object 
 7   sub_plot_shift          634 non-null    object 
 8   GPS_waypt               634 non-null    float64
 9   GPS_id                  634 non-null    float64
 10  GPS                     577 non-null    object 
 11  GPS_latitude            577 non-null    float64
 12  GPS_longitude           577 non-null    float64
 13  GPS_altitude            577 non-null    float64
 14  GPS_precision           577 non-null    fl

In [6]:
if NTV_LITTER_CSV.exists():
    ntv_litter = pd.read_csv(NTV_LITTER_CSV)
else:
    query = f"""
    SELECT 
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.saplings_ntv_litter"""

    # Read the BigQuery table into a dataframe
    ntv_litter = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ntv_litter.to_csv(PLOT_INFO_CSV, index=False)

In [7]:
ntv_litter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   unique_id             674 non-null    object 
 1   count_saplings        589 non-null    float64
 2   litter_bag_weight     620 non-null    float64
 3   litter_sample_weight  620 non-null    float64
 4   ntv_bag_weight        620 non-null    float64
 5   ntv_sample_weight     620 non-null    float64
dtypes: float64(5), object(1)
memory usage: 31.7+ KB


# Calculate carbon stock for litter

In [8]:
# get weight of bag contents
ntv_litter["litter_biomass_kg"] = (
    ntv_litter.ntv_sample_weight - ntv_litter.ntv_bag_weight
) / 1000

In [9]:
litter = ntv_litter[["unique_id", "litter_biomass_kg"]].copy()
litter = vmd0003_eq1(litter, "litter_biomass_kg", 0.15, 0.37)

In [ ]:
litter.rename(
    columns={
        "carbon_stock": "litter_carbon_stock",
        "dry_biomass": "litter_dry_biomass",
    },
    inplace=True,
)

In [24]:
litter.info(), litter.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   unique_id            674 non-null    object 
 1   litter_biomass_kg    620 non-null    float64
 2   litter_dry_biomass   620 non-null    float64
 3   litter_carbon_stock  620 non-null    float64
dtypes: float64(3), object(1)
memory usage: 21.2+ KB


(None,
   unique_id  litter_biomass_kg  litter_dry_biomass  litter_carbon_stock
 0     308D2               0.80               0.120               6.5120
 1     308A2               0.06               0.009               0.4884)

## Export data and upload to BQ

In [25]:
if len(litter) != 0:
    litter.to_csv(CARBON_STOCK_OUTDIR / "litter_carbon_stock.csv", index=False)

In [31]:
# Upload to BQ
table_schema = [
    {"name": "unique_id", "type": "STRING"},
    {"name": "litter_biomass_kg", "type": "FLOAT64"},
    {"name": "litter_dry_biomass", "type": "FLOAT64"},
    {"name": "litter_carbon_stock", "type": "FLOAT64"},
]
if len(litter) != 0:
    pandas_gbq.to_gbq(
        litter,
        f"{DATASET_ID}.litter_carbon_stock",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
        table_schema=table_schema,
    )

100%|██████████| 1/1 [00:00<00:00, 8943.08it/s]


# Calculate carbon stock for non-tree vegetation

In [14]:
ntv_litter["ntv_biomass_kg"] = (
    ntv_litter.ntv_sample_weight - ntv_litter.litter_bag_weight
) / 1000

In [15]:
ntv = ntv_litter[["unique_id", "ntv_biomass_kg"]].copy()
ntv = vmd0003_eq1(ntv, "ntv_biomass_kg", 0.15, 0.37)

In [16]:
ntv.info(), ntv.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   unique_id       674 non-null    object 
 1   ntv_biomass_kg  620 non-null    float64
 2   dry_biomass     620 non-null    float64
 3   carbon_stock    620 non-null    float64
dtypes: float64(3), object(1)
memory usage: 21.2+ KB


(None,
   unique_id  ntv_biomass_kg  dry_biomass  carbon_stock
 0     308D2            0.80        0.120        6.5120
 1     308A2            0.06        0.009        0.4884)

In [17]:
ntv.rename(columns={"carbon_stock": "ntv_carbon_stock"}, inplace=True)

In [18]:
ntv.to_csv(CARBON_STOCK_OUTDIR / "ntv_carbon_stock.csv", index=False)

In [19]:
# Upload to BQ
if len(ntv) != 0:
    pandas_gbq.to_gbq(
        ntv,
        f"{DATASET_ID}.ntv_carbon_stock",
        project_id=GCP_PROJ_ID,
        if_exists=IF_EXISTS,
    )

100%|██████████| 1/1 [00:00<00:00, 9425.40it/s]
